In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from scipy.stats import pearsonr
import seaborn as sns
import scipy.stats as stats
import xgboost as xgb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from sklearn.impute import KNNImputer

In [ ]:
cmr_phenotypes_34K = pd.read_csv('path\\cmr_phenotypes_34496.csv')

In [ ]:
covariates = pd.read_csv('path\\ukb_covariates_34K.csv')

In [ ]:
covariates = covariates.drop(columns='sex')

In [ ]:
sex = pd.read_csv('path\\sex_34496.csv')

In [ ]:
covariates = sex.merge(covariates, on='f.eid', how='left')

In [ ]:
bmi = pd.read_csv('path\\obesity_bmi_height_weight_cmr_phenotypes_45K.csv')
waist = pd.read_csv('path\\waist.tab', sep="\t")
hip = pd.read_csv('path\\hip.tab', sep="\t")

In [ ]:
bmi_ = bmi[['f.eid','BMI']]
waist_ = waist[['f.eid', 'f.48.2.0']]
hip_ = hip[['f.eid', 'f.49.2.0']]

bmi_waist = bmi_.merge(waist_)
bmi_waist_hip = bmi_waist.merge(hip_)

bmi_waist_hip['waist_hip_ratio'] = bmi_waist_hip['f.48.2.0']/bmi_waist_hip['f.49.2.0']

In [ ]:
cmr_covariates = cmr_phenotypes_34K.merge(covariates, on='f.eid', how='left')

In [ ]:
cmr_covariates = cmr_covariates.merge(bmi_waist_hip, on='f.eid')

In [ ]:
def categorize_obesity(bmi):
    if bmi < 18.5:
        return 'underweight'
    elif 18.5 <= bmi < 25:
        return 'normal weight'
    elif 25 <= bmi < 30:
        return 'overweight'
    elif 30 <= bmi < 40:
        return 'obesity'
    else:
        return 'severe obesity'

# Suponiendo que tienes un DataFrame llamado df con la columna 'BMI'
cmr_covariates['obesity_groups'] = cmr_covariates['BMI'].apply(categorize_obesity)

In [ ]:
bmi_waist_hip.isna().sum()

In [ ]:
cmr_covariates.isna().sum()[-15:]

In [ ]:
cmr_covariates['sex'].value_counts()

In [ ]:
female = cmr_covariates[cmr_covariates['sex'] == 0].reset_index(drop=True)
#male = cmr_covariates[cmr_covariates['sex'] == 1].reset_index(drop=True)

In [ ]:
female.isna().sum()

In [ ]:
female

In [ ]:
ethnicity = {
    'white': 1,
    'south_asian': 2,
    'mixed': 3,
    'other': 4,
    'chinese': 5,
    'unknown':6
}

In [ ]:
encoder = LabelEncoder()

In [ ]:
female['ethnicity'] = female['ethnicity'].map(ethnicity)

In [ ]:
female['ethnicity'].value_counts()

In [ ]:
female.columns

# KNN Imputer

In [ ]:
all_cmr_phenotypes_18K = female.drop(columns=['f.eid', 'obesity_groups', 'deprivation_index','highest_qualification_category', 'smoking_status', 'sex', 'drinking_status', 'physical_moderate', 'physical_vigorous', 'sbp', 'BMI', 'f.48.2.0', 'f.49.2.0','waist_hip_ratio'])

In [ ]:
all_cmr_phenotypes_18K

In [ ]:
imputer = KNNImputer(n_neighbors=10)
data_imputed = imputer.fit_transform(all_cmr_phenotypes_18K)

In [ ]:
heart_34K_imputed = pd.DataFrame(data_imputed, columns=all_cmr_phenotypes_18K.columns)

In [ ]:
heart_34K_imputed.columns

In [ ]:
df_knn_ = pd.concat([female[['f.eid','obesity_groups']], heart_34K_imputed ], axis=1)

In [ ]:
df_knn_ #.isna().sum()

# Deconfound_features

In [ ]:
df_knn_cmr_covariates = df_knn_.drop(columns=['age_at_recruitment_visit2', 'obesity_groups']) #.merge(covariates_)

In [ ]:
df_knn_cmr_covariates

In [ ]:
def deconfound_feature(feature, ethnic):
    X = np.array([ethnic]).T
    y = feature  # Dependent variable
    
    model = LinearRegression()
    model.fit(X, y)
    
    predicted = model.predict(X)  # Get predicted values
    residual = y - predicted  # Compute residual
    return residual

In [ ]:
excluded_columns = ['f.eid', 'ethnicity']
cmr_features = [col for col in df_knn_cmr_covariates.columns if col not in excluded_columns]

In [ ]:
len(cmr_features)

In [ ]:
for feature in cmr_features:
    df_knn_cmr_covariates[f"{feature}_deconfounded"] = deconfound_feature(df_knn_cmr_covariates[feature], df_knn_cmr_covariates['ethnicity'])

In [ ]:
df_knn_cmr_covariates.drop(columns=cmr_features + ['f.eid', 'ethnicity'], inplace=True)

In [ ]:
scaler = StandardScaler()

In [ ]:
df_knn_cmr_covariates_ = scaler.fit_transform(df_knn_cmr_covariates)

In [ ]:
df_knn_cmr_covariates_ = pd.DataFrame(df_knn_cmr_covariates_, columns=df_knn_cmr_covariates.columns)

In [ ]:
df_knn_cmr_covariates_

# Prepare Train, validation, testing

In [ ]:
df_knn_cmr_covariates

In [ ]:
df_knn_[['f.eid','obesity_groups','age_at_recruitment_visit2']].merge(df_knn_cmr_covariates_, left_index=True, right_index=True)

In [ ]:
df_merge = df_knn_[['f.eid','obesity_groups','age_at_recruitment_visit2']].merge(df_knn_cmr_covariates, left_index=True, right_index=True)
df_merge['obesity_groups'].value_counts()

In [ ]:
healthy = df_merge[df_merge['obesity_groups'] == 'normal weight'].reset_index(drop=True)
#test_healthy = healthy.sample(n=359, random_state=42).sort_values(by='f.eid').reset_index(drop=True)
train_healthy = healthy #[~healthy['f.eid'].isin(test_healthy['f.eid'])].sort_values(by='f.eid').reset_index(drop=True)

In [ ]:
len(healthy)

In [ ]:
len(train_healthy)

In [ ]:
X = train_healthy.drop(columns=['f.eid', 'obesity_groups', 'age_at_recruitment_visit2'])
y = train_healthy['age_at_recruitment_visit2']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
test_underweight = df_merge[df_merge['obesity_groups'] == 'underweight'].reset_index(drop=True)
test_overweight = df_merge[df_merge['obesity_groups'] == 'overweight'].reset_index(drop=True)
test_obesity = df_merge[df_merge['obesity_groups'] == 'obesity'].reset_index(drop=True)
test_severe_obesity = df_merge[df_merge['obesity_groups'] == 'severe obesity'].reset_index(drop=True)
test_normal_weight = df_merge[df_merge['obesity_groups'] == 'normal weight'].reset_index(drop=True)

In [ ]:
test_normal_weight

In [ ]:
print(len(test_underweight))
print(len(test_normal_weight))
print(len(test_overweight))
print(len(test_obesity))
print(len(test_severe_obesity))

In [ ]:
X_test_underweight = test_underweight.drop(columns=['f.eid', 'obesity_groups', 'age_at_recruitment_visit2'])
y_test_underweight = test_underweight['age_at_recruitment_visit2']
X_test_healthy = test_normal_weight.drop(columns=['f.eid', 'obesity_groups', 'age_at_recruitment_visit2'])
y_test_healthy = test_normal_weight['age_at_recruitment_visit2']
X_test_overweight = test_overweight.drop(columns=['f.eid', 'obesity_groups', 'age_at_recruitment_visit2'])
y_test_overweight = test_overweight['age_at_recruitment_visit2']
X_test_obesity = test_obesity.drop(columns=['f.eid', 'obesity_groups', 'age_at_recruitment_visit2'])
y_test_obesity = test_obesity['age_at_recruitment_visit2']
X_test_severe_obesity = test_severe_obesity.drop(columns=['f.eid', 'obesity_groups', 'age_at_recruitment_visit2'])
y_test_severe_obesity = test_severe_obesity['age_at_recruitment_visit2']

In [ ]:
print('X train', X_train.shape)
print('X val', X_val.shape)
print('Y train', y_train.shape)
print('Y val', y_val.shape)

In [ ]:
y_train.isna().sum()

In [ ]:
X_test_normal_weight = healthy.drop(columns=['f.eid', 'obesity_groups', 'age_at_recruitment_visit2'])
y_test_normal_weight = healthy['age_at_recruitment_visit2']

In [ ]:
print('X test normal weight', X_test_normal_weight.shape)
print('Y test normal weight', y_test_normal_weight.shape)

In [ ]:
print('X test underweight', X_test_underweight.shape)
print('Y test underweight', y_test_underweight.shape)

print('X test healthy', X_test_healthy.shape)
print('Y test healthy', y_test_healthy.shape)

print('X test overweight', X_test_overweight.shape)
print('Y test overweight', y_test_overweight.shape)

print('X test obesity', X_test_obesity.shape)
print('Y test obesity', y_test_obesity.shape)

print('X test severe obesity', X_test_severe_obesity.shape)
print('Y test severe obesity', y_test_severe_obesity.shape)

# Training Model

In [ ]:
def evaluate_model(y_true, y_pred, label="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    corr = np.corrcoef(y_true, y_pred)[0, 1]
    print(f"\n🔹 {label} Results:")
    print(f"   MAE: {mae:.4f}")
    print(f"   MSE: {mse:.4f}")
    print(f"   R²: {r2:.4f}")
    print(f"   Correlation: {corr:.2f}")

## XGBoost

In [ ]:
param_grid = {
    'n_estimators': [10, 20, 50, 100, 200, 500],
    'max_depth': [1,2,3,5],
    'learning_rate': [0.001, 0.01],
    'subsample': [0.01, 0.4, 0.6, 0.8],
    'colsample_bytree': [0.6,0.8],
    'reg_lambda': [0.1,1]  # Regularización L2
}

In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='mae', early_stopping_rounds=50, random_state=1)

In [ ]:
grid_search = GridSearchCV(
    estimator=xgb_model, param_grid=param_grid, 
    scoring='neg_mean_absolute_error', 
    cv=10, verbose=2, n_jobs=-1
)

In [ ]:
grid_search.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

### Bias correction

In [ ]:
y_pred_train = grid_search.best_estimator_.predict(X_train)
y_pred_val = grid_search.best_estimator_.predict(X_val)

In [ ]:
grid_search.best_params_

In [ ]:
# Heart age delta (bias)
heart_age_delta_train = y_pred_train - y_train
heart_age_delta_val = y_pred_val - y_val

In [ ]:
# Train Bias Correction Model
bias_model = LinearRegression()
bias_model.fit(y_train.values.reshape(-1, 1), heart_age_delta_train)

In [ ]:
# Bias correction parameters
beta1 = bias_model.coef_[0]
beta0 = bias_model.intercept_

In [ ]:
beta0

In [ ]:
y_pred_train_corrected = y_pred_train - (beta1 * y_train + beta0)

In [ ]:
y_pred_val_corrected = y_pred_val - (beta1 * y_val + beta0)

In [ ]:
evaluate_model(y_train, y_pred_train, "XGBoost (Before Bias Correction)")

In [ ]:
# Before correction
evaluate_model(y_val, y_pred_val, "XGBoost (Before Bias Correction)")

In [ ]:
# After correction
evaluate_model(y_val, y_pred_val_corrected, "XGBoost (After Bias Correction)")

In [ ]:
# Compute heart age delta before and after correction
heart_age_delta_val_before = y_pred_val - y_val
heart_age_delta_val_after = y_pred_val_corrected - y_val

In [ ]:
corr_pred_actual_before_, _ = pearsonr(y_pred_train, y_train)
corr_pred_actual_after_, _ = pearsonr(y_pred_train_corrected, y_train)

In [ ]:
corr_pred_actual_before, _ = pearsonr(y_pred_val, y_val)
corr_pred_actual_after, _ = pearsonr(y_pred_val_corrected, y_val)

In [ ]:
corr_delta_actual_before, _ = pearsonr(heart_age_delta_val_before, y_val)
corr_delta_actual_after, _ = pearsonr(heart_age_delta_val_after,y_val)

In [ ]:
# Print correlation results
print("\n🔹 Correlation Between Predicted Heart Age & Actual Age:")
print(f"   Before Correction: {corr_pred_actual_before_:.4f}")
print(f"   After Correction: {corr_pred_actual_after_:.4f}")

In [ ]:
# Print correlation results
print("\n🔹 Correlation Between Predicted Heart Age & Actual Age:")
print(f"   Before Correction: {corr_pred_actual_before:.4f}")
print(f"   After Correction: {corr_pred_actual_after:.4f}")

In [ ]:
print("\n🔹 Correlation Between Heart Age Delta & Actual Age:")
print(f"   Before Correction: {corr_delta_actual_before:.4f}")
print(f"   After Correction: {corr_delta_actual_after:.4f}")

In [ ]:
plt.subplot(1, 2, 1)
sns.scatterplot(x=y_val, y=heart_age_delta_val_before, alpha=0.5)
plt.title(f"Before Correction (Corr: {corr_delta_actual_before:.4f})")
plt.xlabel("Actual Age")
plt.ylabel("Heart Age Delta")

In [ ]:
plt.subplot(1, 2, 1)
sns.scatterplot(x=y_val, y=heart_age_delta_val_after, alpha=0.5)
plt.title(f"After Correction (Corr: {corr_delta_actual_after:.4f})")
plt.xlabel("Actual Age")
plt.ylabel("Heart Age Delta")

### Test delta heart

In [ ]:
y_pred_test_underweight = grid_search.best_estimator_.predict(X_test_underweight)
y_pred_test_healthy = grid_search.best_estimator_.predict(X_test_normal_weight)
y_pred_test_overweight = grid_search.best_estimator_.predict(X_test_overweight)
y_pred_test_obesity = grid_search.best_estimator_.predict(X_test_obesity)
y_pred_test_severe_obesity = grid_search.best_estimator_.predict(X_test_severe_obesity)

In [ ]:
y_pred_test_healthy

In [ ]:
# Heart age delta (bias)
heart_age_delta_test_underweight = y_pred_test_underweight - y_test_underweight
heart_age_delta_test_healthy = y_pred_test_healthy - y_test_normal_weight
heart_age_delta_test_overweight = y_pred_test_overweight - y_test_overweight
heart_age_delta_test_obesity = y_pred_test_obesity - y_test_obesity
heart_age_delta_test_severe_obesity = y_pred_test_severe_obesity - y_test_severe_obesity

In [ ]:
y_pred_test_underweight_corrected = y_pred_test_underweight - (beta1 * y_test_underweight + beta0)
y_pred_test_healthy_corrected = y_pred_test_healthy - (beta1 * y_test_normal_weight + beta0)
y_pred_test_overweight_corrected = y_pred_test_overweight - (beta1 * y_test_overweight + beta0)
y_pred_test_obesity_corrected = y_pred_test_obesity - (beta1 * y_test_obesity + beta0)
y_pred_test_severe_obesity_corrected = y_pred_test_severe_obesity - (beta1 * y_test_severe_obesity + beta0)

In [ ]:
heart_age_delta_test_underweight_after = y_pred_test_underweight_corrected - y_test_underweight
heart_age_delta_test_healthy_after = y_pred_test_healthy_corrected - y_test_normal_weight
heart_age_delta_test_overweight_after = y_pred_test_overweight_corrected - y_test_overweight
heart_age_delta_test_obesity_after = y_pred_test_obesity_corrected - y_test_obesity
heart_age_delta_test_severe_obesity_after = y_pred_test_severe_obesity_corrected - y_test_severe_obesity

## Features Importance

In [ ]:
from xgboost import plot_importance

In [ ]:
plt.figure(figsize=(10, 6))
ax = plot_importance(grid_search.best_estimator_, max_num_features=10)
plt.draw()

current_labels = [item.get_text() for item in ax.get_yticklabels()]
new_labels = [label.replace('_', ' ').replace(' deconfounded', '') for label in current_labels]
ax.set_yticklabels(new_labels)

plt.title('Top 10 Most Important Features', fontsize=16)
ax.set_ylabel('Features', fontsize=16)  # <- y-axis title and size
ax.set_xlabel('F-score', fontsize=16)  # <- x-axis title and size

# Color bars based on importance
bars = ax.patches
importances = np.array([bar.get_width() for bar in bars])
norm = plt.Normalize(importances.min(), importances.max())
cmap = plt.cm.viridis

for bar, importance in zip(bars, importances):
    bar.set_color(cmap(norm(importance)))

plt.savefig('\\Users\\Cynthia Maldonado\\OneDrive - Queen Mary, University of London\\Biological Aging\\concept paper\\figures\\feature_importance_female.png', bbox_inches='tight', dpi=300) 

plt.show()

# Analysis between BMI groups

In [ ]:
test_underweight_ = test_underweight[['f.eid', 'obesity_groups', 'age_at_recruitment_visit2']]
test_healthy_ = test_normal_weight[['f.eid', 'obesity_groups', 'age_at_recruitment_visit2']]
test_overweight_ = test_overweight[['f.eid', 'obesity_groups', 'age_at_recruitment_visit2']]
test_obesity_ = test_obesity[['f.eid', 'obesity_groups', 'age_at_recruitment_visit2']]
test_severe_obesity_ = test_severe_obesity[['f.eid', 'obesity_groups', 'age_at_recruitment_visit2']]

In [ ]:
delta_underweight = test_underweight_.merge(heart_age_delta_test_underweight_after, left_index=True, right_index=True)
delta_healthy = test_healthy_.merge(heart_age_delta_test_healthy_after, left_index=True, right_index=True)
delta_overweight = test_overweight_.merge(heart_age_delta_test_overweight_after, left_index=True, right_index=True)
delta_obesity = test_obesity_.merge(heart_age_delta_test_obesity_after, left_index=True, right_index=True)
delta_severe_obesity = test_severe_obesity_.merge(heart_age_delta_test_severe_obesity_after, left_index=True, right_index=True)

In [ ]:
heart_age_delta_test_healthy_after

In [ ]:
delta_obesity_merge = pd.concat([delta_underweight, delta_healthy, delta_overweight, delta_obesity, delta_severe_obesity])

In [ ]:
delta_obesity_merge.sort_values(by='f.eid').reset_index(drop=True)

In [ ]:
delta_obesity_merge['obesity_groups'] = delta_obesity_merge['obesity_groups'].replace('healthy range', 'normal weight')

In [ ]:
delta_obesity_merge['delta'] = delta_obesity_merge['age_at_recruitment_visit2_y']

In [ ]:
delta_obesity_merge = delta_obesity_merge.drop(columns=['age_at_recruitment_visit2_y'])

In [ ]:
delta_obesity_merge

In [ ]:
# Boxplot of Heart Age Delta by obesity groups
sns.boxplot(x=delta_obesity_merge['obesity_groups'], y=delta_obesity_merge['delta'])
plt.title("Distribution of Heart Age Delta for obesity group")
plt.show()

# ANOVA (if there is more than 2 groups)
anova_result = stats.f_oneway(
    delta_obesity_merge[delta_obesity_merge['obesity_groups'] == 'healthy range']['delta'],
    delta_obesity_merge[delta_obesity_merge['obesity_groups'] == 'overweight']['delta'],
    delta_obesity_merge[delta_obesity_merge['obesity_groups'] == 'obesity']['delta'],
    delta_obesity_merge[delta_obesity_merge['obesity_groups'] == 'severe obesity']['delta'],
    delta_obesity_merge[delta_obesity_merge['obesity_groups'] == 'underweight']['delta']
    
)
print("ANOVA p-value:", anova_result.pvalue)


In [ ]:
#Group by obesity and calculate descriptive statistics
desc_stats = delta_obesity_merge.groupby('obesity_groups')['delta'].describe()

# Select only the most relevant metrics
desc_stats = desc_stats[['count', 'mean', 'std', 'min', '50%', 'max']]

# Rename columns for better clarity
desc_stats.columns = ['Muestras', 'Media', 'Desviación Estándar', 'Mínimo', 'Mediana', 'Máximo']

# Display results
print(desc_stats)

In [ ]:
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
p_values_dict= {}

In [ ]:
#Check if there are at least two groups with enough data
group_counts = delta_obesity_merge['obesity_groups'].value_counts()
# ANOVA
groups = [delta_obesity_merge[delta_obesity_merge['obesity_groups'] == group]['delta'] for group in delta_obesity_merge['obesity_groups'].unique()]
anova_result = stats.f_oneway(*groups)
if not (pd.isna(anova_result.statistic) or pd.isna(anova_result.pvalue)):
    print(f"ANOVA - F: {anova_result.statistic:.3f}, p-value: {anova_result.pvalue:.5f}")
    p_values_dict['delta'] = anova_result.pvalue

    print("Significant differences found, applying Tukey HSD...\n")
    tukey_result = pairwise_tukeyhsd(delta_obesity_merge['delta'], delta_obesity_merge['obesity_groups'], alpha=0.05)
    print(tukey_result)
else:
    print("ANOVA could not be calculated due to data issues.\n")

In [ ]:
order = ["underweight", "normal weight", "overweight", "obesity", "severe obesity"]
# Calculate delta mean by obesity groups
delta_means = delta_obesity_merge.groupby("obesity_groups")["delta"].mean()
# Reorganise obesity groups
delta_means = delta_means[order]


plt.figure(figsize=(8, 6))
delta_means.plot(kind="bar", color="skyblue", edgecolor="black", width=0.7)
plt.title("Average Heart Age Delta by Obesity Groups", fontsize=14)
plt.xlabel("Obesity Groups", fontsize=12)
plt.ylabel("Mean Heart Age Delta", fontsize=12)
plt.xticks(rotation=45)
plt.savefig('path\\mean_heart_age_delta_female.png', bbox_inches='tight', dpi=300) 
plt.show()

In [ ]:
for group in delta_obesity_merge['obesity_groups'].unique():
    subset = delta_obesity_merge[delta_obesity_merge['obesity_groups'] == group]
    corr, pval = stats.pearsonr(subset['delta'], subset['age_at_recruitment_visit2_x'])
    print(f"{group} -> Correlación: {corr:.4f}, p-valor: {pval:.4f}")

# Obesity Clusters

In [ ]:
# Values of VAT, ASAT and PAT
obesity_qc = pd.read_csv('path\\obesity_measures.csv')

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
merge_fat_delta = obesity_qc.merge(delta_obesity_merge)

In [ ]:
merge_fat_delta.groupby('obesity_groups')['delta'].describe()

In [ ]:
merge_fat_without_nan = merge_fat_delta.dropna(subset=['f.22407.2.0', 'f.22408.2.0', 'meanArea (cm2)']).reset_index(drop=True)

In [ ]:
features_for_clustering = merge_fat_without_nan[['f.22407.2.0', 'f.22408.2.0', 'meanArea (cm2)']]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_for_clustering)

In [ ]:
# Apply K-Means Clustering
kmeans = KMeans(n_clusters=2, random_state=1)

In [ ]:
merge_fat_without_nan['obesity_cluster'] = kmeans.fit_predict(X_scaled)

In [ ]:
cross_tab = pd.crosstab(merge_fat_without_nan['obesity_groups'], merge_fat_without_nan['obesity_cluster'])
print(cross_tab)

In [ ]:
merge_fat_without_nan.groupby('obesity_cluster')[list(features_for_clustering)].agg(['mean', 'std', 'min', 'max'])

In [ ]:
order = ["underweight", "healthy range", "overweight", "obesity", "severe obesity"]
# Calculate the mean of delta by obesity group
delta_means = merge_fat_without_nan.groupby("obesity_cluster")["delta"].mean()
# Reorder the obesity groups according to the defined order
delta_means = delta_means[order]
plt.figure(figsize=(8, 6))
delta_means.plot(kind="bar", color="skyblue", edgecolor="black", width=0.7)
plt.title("Mean Heart Age Delta by Obesity cluster", fontsize=14)
plt.xlabel("Obesity cluster", fontsize=12)
plt.ylabel("Mean Heart Age Delta", fontsize=12)
plt.xticks(rotation=45)

plt.show()

In [ ]:
# Define the order of the obesity groups
obesity_order = ["underweight", "normal weight", "overweight", "obesity", "severe obesity"]

# Group by cluster and obesity to count occurrences
cluster_obesity_counts = merge_fat_without_nan.groupby(['obesity_cluster', 'obesity_groups']).size().unstack()

# Reorder the columns according to the defined order
cluster_obesity_counts = cluster_obesity_counts[obesity_order]

# Calculate proportions
cluster_proportions = cluster_obesity_counts.div(cluster_obesity_counts.sum(axis=1), axis=0)

# Define the custom color palette
palette_colors = {
    "underweight": "#1f77b4",      # Blue
    "normal weight": "#2ca02c",    # Green
    "overweight": "#ff7f0e",       # Orange
    "obesity": "#d62728",          # Red
    "severe obesity": "#9467bd"    # Purple
}

# Plot
plt.figure(figsize=(10, 6))
cluster_proportions.plot(
    kind="bar", 
    stacked=True, 
    color=[palette_colors[label] for label in obesity_order], 
    alpha=0.8
)
plt.title("Proportion of Obesity Groups in Each Cluster")
plt.ylabel("Proportion")
plt.xlabel("Obesity Cluster")
plt.legend(title="Obesity Groups", bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)
plt.xticks(rotation=0)
plt.show()

In [ ]:
chi2_stat, p_value, dof, expected = stats.chi2_contingency(cluster_obesity_counts.fillna(0))

In [ ]:
print(f"Chi2 Statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:}")

# 🔹 Interpretación
if p_value < 0.05:
    print("There is a significant association between the clusters and the obesity groups (reject H0).")
else:
    print("There is no significant association (fail to reject H0).")

In [ ]:
confounders = ['ethnicity', 'drinking_status', 'physical_moderate', 'smoking_status_numeric', 'age_at_recruitment_visit2']
exposure_vars = ['BMI', 'waist_hip_ratio', 'VAT', 'ASAT', 'pericardial', 'waist', 'hip']
obesity_groups = ['overweight', 'obesity', 'healthy range', 'severe obesity', 'underweight']

In [ ]:
results = []

# Iterar sobre cada grupo en obesity_groups
for group in obesity_groups:
    print(f"\nProcesando grupo: {group}")

    # Filtrar el DataFrame para el grupo actual
    df_group = ml_delta_test_drop_confounders[ml_delta_test_drop_confounders['obesity_groups'] == group].copy()

    # Verificar que el grupo tenga suficientes datos
    if len(df_group) < 10:  # Evitar problemas con grupos muy pequeños
        print(f"Grupo {group} tiene muy pocos datos ({len(df_group)}). Omitido.")
        continue

    # Crear un DataFrame vacío para almacenar exposiciones deconfundadas
    deconfounded_exposures = pd.DataFrame(index=df_group.index)

    # Regresión para cada variable de exposición
    for var in exposure_vars:
        X = sm.add_constant(df_group[confounders])  # Matriz de diseño con confusores
        y = df_group[var]

        # Verificar que no haya NaNs
        if X.isnull().any().any() or y.isnull().any():
            print(f"Saltando {var} en grupo {group} por valores NaN")
            continue

        # Ajustar el modelo de regresión lineal
        model = sm.OLS(y, X).fit()

        # Guardar residuos como exposición deconfundada
        deconfounded_exposures[var] = model.resid

    # Calcular número de pruebas para Bonferroni
    num_tests = len(deconfounded_exposures.columns)

    # Calcular correlaciones con Heart Age Delta
    for var in deconfounded_exposures.columns:
        r, p_value = pearsonr(deconfounded_exposures[var], df_group['delta'])

        # Aplicar corrección de Bonferroni
        bonferroni_p = p_value * num_tests

        # Determinar significancia
        significant = bonferroni_p < 0.05

        # Guardar resultados en la lista
        results.append({
            'Obesity Group': group,
            'Variable': var,
            'Pearson_r': r,
            'P_value': p_value,
            'Bonferroni_P_value': bonferroni_p,
            'Significant': significant
        })

# Convertir la lista en un DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df